In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


class BoneAgeModel(nn.Module):
    def __init__(self):
        super(BoneAgeModel, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.model.classifier = nn.Linear(1280, 1)  

    def forward(self, x):
        return self.model(x).squeeze(1)  

# Define Albumentations Transforms
train_transforms = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensorV2()
])

valid_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensorV2()
])

# Define Custom Dataset
class BoneAgeDataset(Dataset):
    def __init__(self, df, img_folder, transform=None):
        self.df = df
        self.img_folder = img_folder
        self.transform = transform

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_folder, str(self.df.iloc[idx, 0]) + ".png")
        
        # Read as grayscale and convert to RGB
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        image = np.stack([image] * 3, axis=-1)  # Convert grayscale to 3 channels

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]

        label = torch.tensor(self.df.iloc[idx, 1], dtype=torch.float32)  # Bone Age in months
        return image, label

    def __len__(self):
        return len(self.df)

#  Load Data
df = pd.read_csv("/kaggle/input/boneage-dataset/BoneAge_Dataset/BoneAge_train.csv")  # Update path
img_folder = "/kaggle/input/boneage-dataset/BoneAge_Dataset/Training_dataset_Boneage"  # Update path

dataset = BoneAgeDataset(df, img_folder, transform=train_transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_dataset.transform = train_transforms
val_dataset.transform = valid_transforms

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Define Training Function
def train(model, train_loader, val_loader, device, epochs=10, lr=1e-4):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    best_val_loss = float('inf')
    best_model_path = "best_model.pth"

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
        
        val_loss /= len(val_loader)
        scheduler.step()

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with Val Loss: {best_val_loss:.4f}")

# Run Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BoneAgeModel()

train(model, train_loader, val_loader, device, epochs=25, lr=1e-4)


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/m

Epoch [1/25], Train Loss: 15603.3783, Val Loss: 12967.8775
Best model saved with Val Loss: 12967.8775


100%|██████████| 316/316 [02:44<00:00,  1.93it/s]


Epoch [2/25], Train Loss: 12729.8836, Val Loss: 12268.0037
Best model saved with Val Loss: 12268.0037


100%|██████████| 316/316 [02:43<00:00,  1.93it/s]


...

100%|██████████| 316/316 [02:43<00:00,  1.93it/s]


Epoch [5/25], Train Loss: 7512.3327, Val Loss: 5970.7094
Best model saved with Val Loss: 5970.7094


100%|██████████| 316/316 [02:45<00:00,  1.91it/s]


Epoch [6/25], Train Loss: 6516.1617, Val Loss: 5908.9110
Best model saved with Val Loss: 5908.9110


 11%|█         | 34/316 [00:18<02:23,  1.97it/s]

In [ ]:
import torch

# Save the trained model
torch.save(model.state_dict(), "best_model.pth")
print("✅ Best model saved successfully!")


In [ ]:
# Load the saved model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Set model to evaluation mode
print("✅ Model loaded for inference!")


In [ ]:
import torch
import pandas as pd
import cv2
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import models
import torch.nn as nn

# Define the Model Class (Must match trained model)
class BoneAgeModel(nn.Module):
    def __init__(self):
        super(BoneAgeModel, self).__init__()
        self.model = models.efficientnet_b0(pretrained=False)  # No need to download weights again
        self.model.classifier = nn.Linear(1280, 1)  # Regression output

    def forward(self, x):
        return self.model(x).squeeze(1)  # Output shape (batch,)

#  Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BoneAgeModel()
model.load_state_dict(torch.load("/kaggle/working/best_model.pth", map_location=device))
model.to(device)
model.eval()  # Set model to evaluation mode

# Define Albumentations Transforms for Test Images
test_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.5], std=[0.5]),
    ToTensorV2()
])

# Define Custom Test Dataset
class BoneAgeTestDataset(Dataset):
    def __init__(self, img_folder, transform=None):
        self.img_folder = img_folder
        self.transform = transform
        self.image_ids = sorted(os.listdir(img_folder))  # Get all image filenames

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.img_folder, img_id)

        # Read as grayscale and convert to RGB
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        image = np.stack([image] * 3, axis=-1)  # Convert grayscale to 3 channels

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, img_id.replace(".png", "")  

    def __len__(self):
        return len(self.image_ids)


test_folder = "/kaggle/input/boneage-dataset/BoneAge_Dataset/Test_dataset_BoneAge"  # Update path
test_dataset = BoneAgeTestDataset(test_folder, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


predictions = []

with torch.no_grad():
    for images, image_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        outputs = outputs.cpu().numpy()  

        
        for img_id, pred in zip(image_ids, outputs):
            predictions.append((img_id, pred))


submission_df = submission_df[submission_df["ID"] != "BoneAge_test_1742"]

submission_df.to_csv("/kaggle/working/submission.csv", index=False)

print("✅ Submission file saved: /kaggle/working/submission.csv")

